In [87]:
import requests
import lxml.html as lh
import pandas as pd

In [88]:
url='http://jan.ucc.nau.edu/~swilson/C4FullSite/BigTable.html'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [89]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]

In [90]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"[N,i]"
2:"V"
3:"E"
4:"Tr
"
5:"W?"
6:"B?"
7:"|AG|"
8:"vs"
9:"ds"
10:"#STO"
11:"gi"
12:"dm"
13:"NAME
"


In [91]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 13, the //tr data is not from our table 
    if len(T)!=13:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [92]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [93]:
#replace all \n with ''
df = df.replace('\n','', regex=True)

In [94]:
#changing the header names
df.rename(columns={'V':'no. vertices','E':'no. edges', 'Tr\n':'transitivity', 'W?':'worthy','B?':'bipartite',
                  '|AG|':'order of symmetry group', 'vs':'order of a vertex stabilizer',
                  'ds':'order of dart stabilizer', '#STO': 'no. of semi-transitive orientations',
                  'gi':'girth', 'dm': 'dimeter', 'NAME\n':'name'},inplace=True)

In [95]:
df['transitivity'] = df['transitivity'].replace('DT','dart-transitive',regex=True).replace('SS','semisymmetric',regex=True).replace('HT','half-arc-transitive',regex=True)

In [96]:
df['worthy'] = df['worthy'].replace('W','worthy',regex=True).replace('U','unworhty',regex=True)

In [97]:
df['bipartite'] = df['bipartite'].replace('NB','not-bipartite',regex=True).replace('Bip','bipartite',regex=True)

In [98]:
#calculating prime factorzations
df['order of symmetry group'] = df['order of symmetry group'].replace('\^','**',regex=True).replace('\)\(',')*(',regex=True).apply(lambda x:eval(x) if isinstance(x, str) else x)

In [100]:
#calculating prime factorizations
df['order of a vertex stabilizer'] = df['order of a vertex stabilizer'].replace('\^','**',regex=True).replace('\)\(',')*(',regex=True).apply(lambda x:eval(x) if isinstance(x, str) else x)

In [101]:
#calculating prime factorizations
df['order of dart stabilizer'] = df['order of dart stabilizer'].replace('\^','**',regex=True).replace('\)\(',')*(',regex=True).apply(lambda x:eval(x) if isinstance(x, str) else x)

In [103]:
#df.to_csv("C4_table.csv", index=False)